In [ ]:
# %pip install pandas pyarrow
# %pip install tqdm
# %pip install plotly
# %pip install nbformat
# %pip install ipywidgets
# %pip install numba

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import os

# Imports from local scripts.
from data_downloader import download, get_filename
from trailing_stop_loss import generate_stop_loss_levels
from trailing_stop_loss import calculate_pl

In [ ]:
ticker = 'AAPL'
interval = '5s'
year = '2024'

In [ ]:
# get data

filename = get_filename(ticker, interval, year)
# If file doesn't exist, then download it.
if not os.path.exists(filename):
	download(ticker, interval, year)

df = pd.read_parquet(filename)

# # show what is loaded
# fig = px.line(df[-1000:], y='open', title=f'{ticker} Open Prices')
# fig.show()

total_bars = df.shape[0]
print(f'Total bars: {total_bars} ({total_bars:,})')

In [ ]:
# Set trailing stop-loss levels
max_sl = 1
sl_steps = 10
stop_loss_percents, stop_loss_levels = generate_stop_loss_levels(max_sl, sl_steps)

print("Stop-loss levels (in percent):")
print(stop_loss_percents)
print("\nStop-loss levels (as fractions):")
print(stop_loss_levels)

#### helper functions

In [ ]:
import plotly.graph_objects as go
# New cell - Visualization function adapted for calculate_pl results
def plot_sl_profits_2d(df_exp, results_2d, stop_loss_levels, start_idx=0):
    """Plot trailing SL profits using 2D array results from calculate_pl.
    
    Args:
        df_exp: OHLC DataFrame
        results_2d: 2D NumPy array from calculate_pl [sl_levels, entry_points]
        stop_loss_levels: Array of SL levels
        start_idx: Starting index offset for plotting
    """
    fig = go.Figure()
    
    # Prepare data for scatter plot
    sl_indices = []
    entry_indices = []
    profits = []
    sl_prices = []
    
    for sl_idx, sl_level in enumerate(stop_loss_levels):
        if sl_level == 0:
            continue
            
        # Get profits for this SL level
        sl_profits = results_2d[sl_idx, :]
        
        # Create corresponding entry indices and SL prices
        for entry_idx in range(len(sl_profits)):
            if not np.isnan(sl_profits[entry_idx]):
                sl_indices.append(sl_idx)
                entry_indices.append(entry_idx + start_idx)  # Adjust for plot offset
                profits.append(sl_profits[entry_idx])
                
                # Calculate SL price: entry_price * (1 + sl_level)
                entry_price = df_exp.iloc[entry_idx]['open']
                sl_price = entry_price * (1 + sl_level)
                sl_prices.append(sl_price)
    
    # Convert to arrays for plotting
    profits = np.array(profits)
    sl_prices = np.array(sl_prices)
    entry_indices = np.array(entry_indices)
    
    if len(profits) == 0:
        print("No valid data to plot")
        return
    
    # Calculate color scale
    min_profit = profits.min()
    max_profit = profits.max()
    zero_position = (0 - min_profit) / (max_profit - min_profit) if max_profit != min_profit else 0.5
    
    custom_colorscale = [
        [0.0, 'red'],           # Minimum profit = red
        [zero_position, 'white'], # Zero profit = white
        [1.0, 'blue']           # Maximum profit = blue
    ]
    
    # Add stop-loss points
    fig.add_trace(go.Scatter(
        x=entry_indices,
        y=sl_prices,
        mode='markers',
        marker=dict(
            size=3,
            color=profits,
            colorscale=custom_colorscale,
            showscale=True,
            colorbar=dict(title='Profit (%)'),
            cmin=min_profit,
            cmax=max_profit
        ),
        name='SL Points'
    ))
    
    # Add OHLC prices on top
    x_indices = np.arange(len(df_exp)) + start_idx
    
    fig.add_trace(go.Scatter(
        x=x_indices,
        y=df_exp['open'],
        mode='lines',
        line=dict(color='black', width=1),
        name='Open'
    ))
    fig.add_trace(go.Scatter(
        x=x_indices,
        y=df_exp['high'],
        mode='lines',
        line=dict(color='black', width=1),
        name='High'
    ))
    fig.add_trace(go.Scatter(
        x=x_indices,
        y=df_exp['low'],
        mode='lines',
        line=dict(color='black', width=1),
        name='Low'
    ))
    
    fig.update_layout(
        title="Trailing SL profits (using calculate_pl)",
        showlegend=False
    )
    
    fig.show()

In [ ]:
# New cell - Box plot function adapted for calculate_pl results
def box_2d(results_2d, stop_loss_levels, title=None, percentiles=[0, 10, 25, 50, 75, 90, 100]):
    """
    Custom box-like plot function for 2D array results from calculate_pl.
    
    Parameters:
    results_2d: 2D NumPy array from calculate_pl [sl_levels, entry_points]
    stop_loss_levels: Array of SL levels
    title: str, plot title
    percentiles: list of ints, percentiles to display (e.g., [0,10,25,50,75,90,100])
    """
    # Ensure percentiles are sorted and converted to fractions
    percentiles_sorted = sorted(percentiles)
    quantile_fracs = [p / 100 for p in percentiles_sorted]
    
    # Prepare data for plotting
    sl_levels_plot = []
    percentile_values = {p: [] for p in percentiles_sorted}
    mean_values = []
    
    for sl_idx, sl_level in enumerate(stop_loss_levels):
        if sl_level == 0:
            continue
            
        # Get profits for this SL level, excluding NaN values
        sl_profits = results_2d[sl_idx, :]
        valid_profits = sl_profits[~np.isnan(sl_profits)]
        
        if len(valid_profits) == 0:
            continue
            
        sl_levels_plot.append(sl_level)
        
        # Calculate percentiles
        for p, frac in zip(percentiles_sorted, quantile_fracs):
            percentile_value = np.percentile(valid_profits, p)
            percentile_values[p].append(percentile_value)
        
        # Calculate mean
        mean_values.append(np.mean(valid_profits))
    
    # Convert to arrays
    sl_levels_plot = np.array(sl_levels_plot)
    mean_values = np.array(mean_values)
    
    # Initialize figure
    fig = go.Figure()
    
    # Add a trace for each percentile
    for p in percentiles_sorted:
        fig.add_trace(go.Scatter(
            x=sl_levels_plot,
            y=percentile_values[p],
            mode='lines+markers',
            name=f'{p}th percentile',
            line=dict(dash='solid' if p == 50 else 'dash'),
            marker=dict(size=6)
        ))
    
    # Add mean as a bold black line
    fig.add_trace(go.Scatter(
        x=sl_levels_plot,
        y=mean_values,
        mode='lines',
        name='Mean',
        line=dict(color='black', width=4),
        hovertemplate='Mean: %{y:.2f}<extra></extra>'
    ))
    
    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title='sl_level',
        yaxis_title='profit',
        legend_title='Metrics',
        template='plotly_white'
    )
    
    return fig

#### calculate profits

In [ ]:
# New cell - Calculate results using calculate_pl
tsl_profits = calculate_pl(df, stop_loss_levels)
print(f"Results shape: {tsl_profits.shape}")
print(f"Shape interpretation: ({len(stop_loss_levels)} SL levels, {len(df)-1} entry points)")

#### show results

In [ ]:
# New cell - Test visualization with calculate_pl results
plot_shift = 75*1000
plot_window = 10*1000

# Slice the price data
plot_df_exp = df[plot_shift:plot_shift + plot_window]

# Slice the 2D results array (columns correspond to entry points)
plot_results_2d = tsl_profits[:, plot_shift:plot_shift + plot_window]

print(f"Plotting window: {plot_shift} to {plot_shift + plot_window}")
print(f"Plot data shape: {plot_df_exp.shape}")
print(f"Plot results shape: {plot_results_2d.shape}")

# Create visualization
plot_sl_profits_2d(plot_df_exp, plot_results_2d, stop_loss_levels, start_idx=plot_shift)

In [ ]:
# New cell - Second visualization example (matching the second commented cell)
plot_shift = 85*1000
plot_window = 10*1000

# Slice the price data
plot_df_exp = df[plot_shift:plot_shift + plot_window]

# Slice the 2D results array
plot_results_2d = tsl_profits[:, plot_shift:plot_shift + plot_window]

print(f"Plotting window: {plot_shift} to {plot_shift + plot_window}")
print(f"Plot data shape: {plot_df_exp.shape}")
print(f"Plot results shape: {plot_results_2d.shape}")

# Create visualization
plot_sl_profits_2d(plot_df_exp, plot_results_2d, stop_loss_levels, start_idx=plot_shift)

In [ ]:
# New cell - Example usage of box plot with calculate_pl results
fig = box_2d(tsl_profits, stop_loss_levels,
    title='Profit Distribution by Stop-Loss Level (calculate_pl)',
    percentiles=[0, 25, 50, 75, 90, 100])
fig.show()